In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-7b-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-7b-instruct", trust_remote_code=True, torch_dtype=torch.bfloat16).cuda()


messages=[
    { 'role': 'user', 'content': "Сгенерируй код сортировки пузырьком на Python"}
]

inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
# tokenizer.eos_token_id is the id of <|EOT|> token
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Вот код сортировки пузырьком на Python:

```python
def bubble_sort(nums):
    # Устанавливаем флаг swapped в True, чтобы цикл запустился хотя бы один раз
    swapped = True
    while swapped:
        swapped = False
        for i in range(len(nums) - 1):
            if nums[i] > nums[i + 1]:
                # Меняем элементы
                nums[i], nums[i + 1] = nums[i + 1], nums[i]
                # Устанавливаем флаг swapped
                swapped = True


# Тестируем код
nums = [64, 34, 25, 12, 22, 11, 90]
bubble_sort(nums)
print(nums)
```

В этом коде мы проходим по списку и сравниваем каждый элемент со следующим. Если они упорядочены неправильно, мы меняем их местами. Мы повторяем этот процесс до тех пор, пока не пройдем по списку без изменений, то есть когда флаг `swapped` останется `False`.



In [2]:
print(1)

1


In [3]:
model.device

device(type='cuda', index=0)

In [4]:
import numpy as np 
print(f'num params before SVD compression = {sum( [ np.prod(item.size())  for item in model.parameters() ])}')

num params before SVD compression = 6740512768


In [5]:
31 // 8

3

In [7]:
import sys
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM


class LaserLinear(torch.nn.modules.module.Module):

    def __init__(self, weight: torch.Tensor, ratio: float):
        super().__init__()
        
        n , m = weight.shape
        U, S, VT = torch.linalg.svd(weight, full_matrices=False)
        
        with torch.inference_mode(): 

            n_compress = torch.where( torch.cumsum(S, dim = 0) / S.sum() > ratio, 0, 1).sum()
            
            #for trsh in [0.5, 0.6, 0.7, 0.8, 0.9]: 
            #    print(f'n_significant values = { torch.where( torch.cumsum(S, dim = 0) / S.sum() > trsh, 0, 1).sum() } out of {len(S)} for trsh = {trsh}')

        self.applied_svd = False
        
        if n * n_compress + n_compress * m < n * m:  
        
        
            truc_u, truc_s, truc_v = torch.svd_lowrank(weight.float(), q=n_compress, niter=3)
            
        #truc_sigma = torch.diag(truc_s)
        
        #print(f'truc_u.shape =  {truc_u.shape}')
        #print(f'truc_v.shape =  {truc_v.shape}')
        #print(f'truc_s.shape =  {truc_s.shape}')
        
        #### Replace Attn, MLP ####
            
        #sqrtSigma = torch.sqrt(truc_sigma)
        #U = torch.matmul(truc_u, sqrtSigma)#.cpu().to(dtype)
        #V = torch.matmul(sqrtSigma, truc_v)#.cpu().to(dtype)
            
            print(f'Applying SVD')
            print(f'Num params after = {n * n_compress + n_compress * m}')
            print(f'Num params before = {n * m}')
            
            self.U = torch.nn.Parameter(truc_u)#.type(weight.dtype)
            self.sigma = torch.nn.Parameter(truc_s)#.type(weight.dtype)
            self.V = torch.nn.Parameter(truc_v)#.type(weight.dtype)
            self.applied_svd = True
        else: 
            self.W = torch.nn.Parameter(weight)
        #print(f'self.U.shape =  {self.U.shape}')
        #print(f'self.sigma.shape =  {self.sigma.shape}')
        #print(f'self.V.shape =  {self.V.shape}') 
        
        #print(f'Mat shape = {(self.U @ self.sigma @ self.V).shape}')
        #print()
        
    def forward(self, input: torch.Tensor) -> torch.Tensor:
        if self.applied_svd: 
            return input @ (self.U @ torch.diag(self.sigma) @ self.V.T).T
        else: 
            return input @ (self.W).T

In [ ]:
import sys
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM


class LaserLinear(torch.nn.modules.module.Module):

    def __init__(self, weight: torch.Tensor, ratio: float):
        super().__init__()
        
        n , m = weight.shape
        U, S, VT = torch.linalg.svd(weight, full_matrices=False)
        
        with torch.inference_mode(): 

            n_compress = torch.where( torch.cumsum(S, dim = 0) / S.sum() > ratio, 0, 1).sum()
            
            #for trsh in [0.5, 0.6, 0.7, 0.8, 0.9]: 
            #    print(f'n_significant values = { torch.where( torch.cumsum(S, dim = 0) / S.sum() > trsh, 0, 1).sum() } out of {len(S)} for trsh = {trsh}')

        self.applied_svd = False
        
        if n * n_compress + n_compress * m < n * m:  
        
        
            truc_u, truc_s, truc_v = torch.svd_lowrank(weight.float(), q=n_compress, niter=3)
            
        #truc_sigma = torch.diag(truc_s)
        
        #print(f'truc_u.shape =  {truc_u.shape}')
        #print(f'truc_v.shape =  {truc_v.shape}')
        #print(f'truc_s.shape =  {truc_s.shape}')
        
        #### Replace Attn, MLP ####
            
        #sqrtSigma = torch.sqrt(truc_sigma)
        #U = torch.matmul(truc_u, sqrtSigma)#.cpu().to(dtype)
        #V = torch.matmul(sqrtSigma, truc_v)#.cpu().to(dtype)
            
            print(f'Applying SVD')
            print(f'Num params after = {n * n_compress + n_compress * m}')
            print(f'Num params before = {n * m}')
            
            self.U = torch.nn.Parameter(truc_u)#.type(weight.dtype)
            self.sigma = torch.nn.Parameter(truc_s)#.type(weight.dtype)
            self.V = torch.nn.Parameter(truc_v)#.type(weight.dtype)
            self.applied_svd = True
        else: 
            self.W = torch.nn.Parameter(weight)
        #print(f'self.U.shape =  {self.U.shape}')
        #print(f'self.sigma.shape =  {self.sigma.shape}')
        #print(f'self.V.shape =  {self.V.shape}') 
        
        #print(f'Mat shape = {(self.U @ self.sigma @ self.V).shape}')
        #print()
        
    def forward(self, input: torch.Tensor) -> torch.Tensor:
        if self.applied_svd: 
            return input @ (self.U @ torch.diag(self.sigma) @ self.V.T).T
        else: 
            return input @ (self.W).T

In [8]:
mlp_names = [
    "mlp.down_proj",
    "mlp.up_proj",
    "mlp.gate_proj",
    "self_attn.q_proj",
    "self_attn.k_proj",
    "self_attn.v_proj",
    "self_attn.o_proj",
]
sys.stdout.flush()

ratio_lst = [1, 0.85, 0.6,  0.2]

treshold = 100
curr_compressed = 0

for layer_index in range(len(model.model.layers)):
    print(f'Processing layer {layer_index}')
    for mlp_name in mlp_names:
        
        original_linear = eval(f"model.model.layers[layer_index].{mlp_name}")
        weight = original_linear.weight.float()
        
            #print(f'weight type = {weight.dtype}')
        exec(f"model.model.layers[layer_index].{mlp_name} = LaserLinear(weight, ratio_lst[layer_index // 8])")
        sys.stdout.flush()
        #if curr_compressed >= 5:  
        #    ratio += 0.1
        #curr_compressed += 1
    #torch.save(model.state_dict(), f"mistral-7b-instruct-laser-{ratio}")

Processing layer 0
Processing layer 1
Processing layer 2
Processing layer 3
Processing layer 4
Processing layer 5
Processing layer 6
Processing layer 7
Processing layer 8
Applying SVD
Num params after = 44587008
Num params before = 45088768
Applying SVD
Num params after = 44692736
Num params before = 45088768
Applying SVD
Num params after = 44360448
Num params before = 45088768
Processing layer 9
Applying SVD
Num params after = 44722944
Num params before = 45088768
Applying SVD
Num params after = 44843776
Num params before = 45088768
Applying SVD
Num params after = 44451072
Num params before = 45088768
Processing layer 10
Applying SVD
Num params after = 44753152
Num params before = 45088768
Applying SVD
Num params after = 44858880
Num params before = 45088768
Applying SVD
Num params after = 44420864
Num params before = 45088768
Processing layer 11
Applying SVD
Num params after = 44526592
Num params before = 45088768
Applying SVD
Num params after = 44571904
Num params before = 45088768


In [9]:
!nvidia-smi

Tue Aug  6 23:53:57 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:83:00.0 Off |                    0 |
| N/A   46C    P0            104W /  400W |   80936MiB /  81920MiB |     89%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): LaserLinear()
          (k_proj): LaserLinear()
          (v_proj): LaserLinear()
          (o_proj): LaserLinear()
          (rotary_emb): LlamaLinearScalingRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): LaserLinear()
          (up_proj): LaserLinear()
          (down_proj): LaserLinear()
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=32256, bias=False)
)

In [11]:
import numpy as np 
print(f'num params after SVD compression = {sum( [ np.prod(item.size())  for item in model.parameters() ])}')

num params after SVD compression = 4724362647


In [12]:
model.to('cuda')

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): LaserLinear()
          (k_proj): LaserLinear()
          (v_proj): LaserLinear()
          (o_proj): LaserLinear()
          (rotary_emb): LlamaLinearScalingRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): LaserLinear()
          (up_proj): LaserLinear()
          (down_proj): LaserLinear()
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=32256, bias=False)
)

In [13]:
model.type(torch.bfloat16)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): LaserLinear()
          (k_proj): LaserLinear()
          (v_proj): LaserLinear()
          (o_proj): LaserLinear()
          (rotary_emb): LlamaLinearScalingRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): LaserLinear()
          (up_proj): LaserLinear()
          (down_proj): LaserLinear()
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=32256, bias=False)
)

In [14]:
inputs

tensor([[32013,  2042,   417,   274, 20926, 14244, 20391,    11, 26696,   254,
         20676, 30742,   339,  8589,  2008,    11,  6908,   457, 20676, 30742,
          7958,    11,   285,   340,   885,  3495,  4301,  4512,   276,  4531,
          8214,    13,  1487,  4636,  2223, 13143,  4301,    11,  5411,   285,
         13936,  4447,    11,   285,   746,  2159,    12, 13517,   250,  8214,
          4301,    11,   340,   540, 20857,   276,  3495,   185, 13518,  3649,
          3475,    25,   185,  3229,   711,  9703,   762,  2613,   949,  1724,
           489,   560,  2393,  1307, 15626,  2296, 15036,   607, 14091,   342,
          5307,  1635,   576,   616, 13003,   185, 13518, 21289,    25,   185]],
       device='cuda:0')

In [15]:
model.device

device(type='cuda', index=0)

In [16]:
messages=[
    { 'role': 'user', 'content': "Write code for sorting array in Python"}
]
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
# tokenizer.eos_token_id is the id of <|EOT|> token
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Sure Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa Papa 

In [ ]:
1 / 6.7

In [17]:
!nvidia-smi 


Wed Aug  7 00:07:14 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:83:00.0 Off |                    0 |
| N/A   30C    P0             68W /  400W |   50292MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
